# Create an Azure Workspace Data asset from local file

## Step 1: Import a Data assets into the Azure ML Workspace using CLI

Import local data into the Azure ML Platform workspace

In [ ]:
%%writefile dataimport.yml
$schema: https://azuremlschemas.azureedge.net/latest/data.schema.json
name: credit_cards
description: Data asset created from local file.
type: uri_file
path: data/default_of_credit_card_clients.csv

Overwriting dataimport.yml


In [ ]:
!az ml data create --file dataimport.yml 

{
  "creation_context": {
    "created_at": "2023-10-25T02:45:34.988353+00:00",
    "created_by": "alex zeltov",
    "created_by_type": "User",
    "last_modified_at": "2023-10-25T02:45:35.009952+00:00"
  },
  "description": "Data asset created from local file.",
  "id": "/subscriptions/f1ea6ed8-82f3-416d-881b-8b376218bc85/resourceGroups/rg_aml/providers/Microsoft.MachineLearningServices/workspaces/aml-default/data/credit_cards/versions/2",
  "name": "credit_cards",
  "path": "azureml://subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourcegroups/SandboxML/workspaces/quick-start-tutorial/datastores/workspaceblobstore/paths/LocalUpload/4b1dfc4d12429b46389cabdf25b886a2/default_of_credit_card_clients.csv",
  "properties": {},
  "resourceGroup": "SandboxML",
  "tags": {},
  "type": "uri_file",
  "version": "2"
}


Read [Access data from Azure cloud storage during interactive development](how-to-access-data-interactive.md) to learn more about data access in a notebook.

## Step 2: Create Parquet version of the Data asset

Create a new version of the data asset and store in Parquet file format

You might have noticed that the data needs a little light cleaning, to make it fit to train a machine learning model. It has:

* two headers
* a client ID column; we wouldn't use this feature in Machine Learning
* spaces in the response variable name

Also, compared to the CSV format, the Parquet file format becomes a better way to store this data. Parquet offers compression, and it maintains schema. Therefore, to clean the data and store it in Parquet, use:

### Step 2a: Install data wrangling libraries

In [5]:
!pip install pyarrow pandas

In [6]:
import pandas as pd

# read in data again, this time using the 2nd row as the header
df = pd.read_csv("data/default_of_credit_card_clients.csv", header=1)
# rename column
df.rename(columns={"default payment next month": "default"}, inplace=True)
# remove ID column
df.drop("ID", axis=1, inplace=True)

# write file to filesystem
df.to_parquet("./data/cleaned-credit-card.parquet")

This table shows the structure of the data in the original **default_of_credit_card_clients.csv** file .CSV file downloaded in an earlier step. The uploaded data contains 23 explanatory variables and 1 response variable, as shown here:

|Column Name(s) | Variable Type  |Description  |
|---------|---------|---------|
|X1     |   Explanatory      |    Amount of the given credit (NT dollar): it includes both the individual consumer credit and their family (supplementary) credit.    |
|X2     |   Explanatory      |   Gender (1 = male; 2 = female).      |
|X3     |   Explanatory      |   Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).      |
|X4     |   Explanatory      |    Marital status (1 = married; 2 = single; 3 = others).     |
|X5     |   Explanatory      |    Age (years).     |
|X6-X11     | Explanatory        |  History of past payment. We tracked the past monthly payment records (from April to September  2005). -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above.      |
|X12-17     | Explanatory        |  Amount of bill statement (NT dollar) from April to September  2005.      |
|X18-23     | Explanatory        |  Amount of previous payment (NT dollar) from April to September  2005.      |
|Y     | Response        |    Default payment (Yes = 1, No = 0)     |

Next, create a new _version_ of the data asset (the data automatically uploads to cloud storage):

> [!NOTE]
>
> This Python code cell sets **name** and **version** values for the data asset it creates. As a result, the code in this cell will fail if executed more than once, without a change to these values. Fixed **name** and **version** values offer a way to pass values that work for specific situations, without concern for auto-generated or randomly-generated values.


### Step 2b: Import local data into the Azure ML Platform workspace

In [11]:
%%writefile dependencies/dataimport_parquet.yml
$schema: https://azuremlschemas.azureedge.net/latest/data.schema.json
name: credit_cards_parquet
description: Data asset created from local file.
type: uri_file
path: ../data/cleaned-credit-card.parquet

Overwriting dependencies/dataimport_parquet.yml


In [12]:
!az ml data create --file dependencies/dataimport_parquet.yml 

{
  "creation_context": {
    "created_at": "2023-10-25T13:43:54.759664+00:00",
    "created_by": "Anton Slutsky",
    "created_by_type": "User",
    "last_modified_at": "2023-10-25T13:43:54.770426+00:00"
  },
  "description": "Data asset created from local file.",
  "id": "/subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourceGroups/SandboxML/providers/Microsoft.MachineLearningServices/workspaces/quick-start-tutorial/data/credit_cards_parquet/versions/1",
  "name": "credit_cards_parquet",
  "path": "azureml://subscriptions/781b03e7-6eb7-4506-bab8-cf3a0d89b1d4/resourcegroups/SandboxML/workspaces/quick-start-tutorial/datastores/workspaceblobstore/paths/LocalUpload/56b293e476b6312818bef5931254dae9/cleaned-credit-card.parquet",
  "properties": {},
  "resourceGroup": "SandboxML",
  "tags": {},
  "type": "uri_file",
  "version": "1"
}



Uploading cleaned-credit-card.parquet (< 1 MB): 0.00B [00:00, ?B/s]
Uploading cleaned-credit-card.parquet (< 1 MB): 100%|##########| 1.58M/1.58M [00:00<00:00, 3.16MB/s]
Uploading cleaned-credit-card.parquet (< 1 MB): 100%|##########| 1.58M/1.58M [00:00<00:00, 3.15MB/s]


